In [1]:
import math
import torch
from torch import nn
from torch.nn import functional as F


In [84]:
exp_capacity=4
indexes=torch.tensor([[0,1,6],[1,2,0]]).unsqueeze(0)
router_probs = torch.zeros(1, 2, 10)
router_probs[0, 0, [0,1,6]] = torch.tensor([0.7, 0.6, 0.2])
router_probs[0, 1, [1,2,0]] = torch.tensor([0.8, 0.5, 0.3])
exp_mask = F.one_hot(indexes, num_classes=10)  # [B, T, k, n_exp]
exp_mask = exp_mask.view(indexes.size(0)*indexes.size(1), 3, 10)  # [B * T, k, n_exp]
exp_mask = exp_mask.permute(1, 0, 2)  # [k, B * T, n_exp] #This permutation is done to order the expert selected for their importance (rank) for each token
# For each token contains the indexes of the top-k experts in one hot encoding
# For example, exp_mask[0] contains the "best" expert selected for each token, while exp_mask[K] the less important expert. This ordering is important to prioritize experts later
exp_rank = exp_mask.reshape(3 * indexes.size(0)*indexes.size(1), 10)  # [k * B * T, n_exp]
exp_rank = torch.cumsum(exp_rank, dim=0) - 1  # [k * B * T, n_exp]
exp_rank = exp_rank.reshape(3,  indexes.size(0)*indexes.size(1), 10)  # [k, B * T, n_exp]
exp_rank.shape

torch.Size([3, 2, 10])

In [85]:
exp_mask *= torch.lt(exp_rank, exp_capacity) # [k, B * T, n_exp]
used_capacity = torch.sum(exp_mask, dim=(0, 1)) # [n_exp]
exp_rank = torch.sum(exp_mask * exp_rank, dim=-1) # [k, B * T]
exp_rank
router_probs = router_probs.view(indexes.size(0)*indexes.size(1),10)[None, :] # [1, B * T, n_exp]
exp_weights = exp_mask * router_probs # [k, B * T, n_exp]
exp_rank_sc = F.one_hot(exp_rank, num_classes=exp_capacity) # [k, B * T, exp_capacity]
exp_rank_sc
cb_weight = torch.sum(exp_weights.unsqueeze(3) * exp_rank_sc.unsqueeze(2), dim=0)
sec_mask = cb_weight.bool() # binary mask of selected experts for each token
#used_capacity
exp_mask

tensor([[[1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [0, 1, 0, 0, 0, 0, 0, 0, 0, 0]],

        [[0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]],

        [[0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
         [1, 0, 0, 0, 0, 0, 0, 0, 0, 0]]])

In [90]:
exp_mask.permute(1,2,0).shape

torch.Size([2, 10, 3])

In [40]:
exp_rank

tensor([[0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
        [1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
        [0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
        [0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
        [0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
        [0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
        [0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
        [0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 1]])

In [37]:
exp_rank

tensor([[-1, -1, -1, -1, -1,  0, -1, -1, -1, -1],
        [ 0, -1, -1, -1, -1,  0, -1, -1, -1, -1],
        [ 0, -1, -1, -1,  0,  0, -1, -1, -1, -1],
        [ 0, -1, -1, -1,  0,  0,  0, -1, -1, -1],
        [ 0,  0, -1, -1,  0,  0,  0, -1, -1, -1],
        [ 0,  1, -1, -1,  0,  0,  0, -1, -1, -1],
        [ 0,  1, -1, -1,  0,  0,  0, -1,  0, -1],
        [ 0,  2, -1, -1,  0,  0,  0, -1,  0, -1],
        [ 0,  3, -1, -1,  0,  0,  0, -1,  0, -1],
        [ 0,  4, -1, -1,  0,  0,  0, -1,  0, -1],
        [ 0,  4, -1,  0,  0,  0,  0, -1,  0, -1],
        [ 0,  4,  0,  0,  0,  0,  0, -1,  0, -1],
        [ 0,  4,  1,  0,  0,  0,  0, -1,  0, -1],
        [ 0,  4,  1,  1,  0,  0,  0, -1,  0, -1],
        [ 0,  5,  1,  1,  0,  0,  0, -1,  0, -1],
        [ 0,  5,  1,  1,  0,  0,  0,  0,  0, -1],
        [ 0,  5,  1,  1,  0,  0,  0,  0,  1, -1],
        [ 0,  5,  2,  1,  0,  0,  0,  0,  1, -1],
        [ 0,  5,  2,  1,  0,  0,  1,  0,  1, -1],
        [ 0,  5,  2,  1,  0,  0,  1,  0,  1,  0],


In [30]:
exp_rank

tensor([[[-1, -1, -1, -1, -1,  0, -1, -1, -1, -1],
         [ 0, -1, -1, -1, -1,  0, -1, -1, -1, -1],
         [ 0, -1, -1, -1,  0,  0, -1, -1, -1, -1],
         [ 0, -1, -1, -1,  0,  0,  0, -1, -1, -1],
         [ 0,  0, -1, -1,  0,  0,  0, -1, -1, -1],
         [ 0,  1, -1, -1,  0,  0,  0, -1, -1, -1],
         [ 0,  1, -1, -1,  0,  0,  0, -1,  0, -1],
         [ 0,  2, -1, -1,  0,  0,  0, -1,  0, -1]],

        [[ 0,  3, -1, -1,  0,  0,  0, -1,  0, -1],
         [ 0,  4, -1, -1,  0,  0,  0, -1,  0, -1],
         [ 0,  4, -1,  0,  0,  0,  0, -1,  0, -1],
         [ 0,  4,  0,  0,  0,  0,  0, -1,  0, -1],
         [ 0,  4,  1,  0,  0,  0,  0, -1,  0, -1],
         [ 0,  4,  1,  1,  0,  0,  0, -1,  0, -1],
         [ 0,  5,  1,  1,  0,  0,  0, -1,  0, -1],
         [ 0,  5,  1,  1,  0,  0,  0,  0,  0, -1]],

        [[ 0,  5,  1,  1,  0,  0,  0,  0,  1, -1],
         [ 0,  5,  2,  1,  0,  0,  0,  0,  1, -1],
         [ 0,  5,  2,  1,  0,  0,  1,  0,  1, -1],
         [ 0,  5,  2,  1,  

In [98]:
b=F.one_hot(torch.Tensor(a).to(torch.long),num_classes=10)
b=b.view(8,3,10)
b=b.permute(1,0,2) #k,b*t,exp
c=b.reshape(3*8,10)
c=torch.cumsum(c,dim=0)-1
c=c.reshape(3,8,10)
c.shape
b*=torch.lt(c,2)
b

tensor([[[0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
         [1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
         [0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
         [0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
         [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],

        [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
         [0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 1, 0, 0]],

        [[0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
         [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
         [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0, 0, 1]]])

In [ ]:
exp_rank = exp_mask.reshape(self.top_k * num_tokens, self.n_exp)  # [k * B * T, n_exp]
exp_rank = torch.cumsum(exp_rank, dim=0) - 1  # cumulative sum of expert selections [k * B * T, n_exp]
exp_rank = exp_rank.reshape(self.top_k, num_tokens, self.n_exp)  # [k, B * T, n_exp]


In [ ]:
class Router(nn.Module):
    def __init__(self,hidden_size,n_experts):
        self.linear_projection=nn.Linear(hidden_size,n_experts)
    def forward(x):
        
        ctx = nullcontext() if not self.router_use_full_prec else torch.amp.autocast(device_type=self.device, enabled=False)
        with ctx: # It's better to force the router to run in fp32 to avoid round errors from exponential
            logits=self.linear_projection(x)
            top_k_logits, top_k_indices = logits.topk(self.top_k, dim=-1) # B,S,K
            # Executing the softmax only on the top_k logits, masking the rest
            router_probs=torch.full_like(logits,float('-inf')) #B,S,N_E
            router_probs.scatter_(-1,top_k_indices,top_k_logits)
            router_probs=F.softmax(router_probs,dim=-1)
            """
            Experts' capacity is computed according to the formula:
            capacity=(K*B*S/N)*cap_factor
            where:
                K = Active experts
                B = Batch size
                S = Sequence length
                N = Total number of expers
                cap_factor = capacity factor (ex. 1.25 for training, 2 for inference)
            """            
            expert_capacity=math.floor(k*cap_factor*num_tokens)/num_exp
            expert_capacity+=capacity%2 # Capacity must be even
            expert_capacity=int(expert_capacity)
            
            

In [10]:
def compute_capacity(num_tokens):
    """
    An helper function to compute experts' capacity according to the formula:
    capacity=(K*B*S/N)*cap_factor
    where:
        K = Active experts
        B = Batch size
        S = Sequence length
        N = Total number of expers
        cap_factor = capacity factor (ex. 1.25 for training, 2 for inference)
    """
    cap_factor=1.25
    k=1
    num_exp=8
    capacity=math.floor(k*cap_factor*num_tokens)/num_exp
    """
        capacity_factor = self.train_capacity if self.training else self.eval_capacity
        capacity = math.floor(self.top_k * capacity_factor * tokens_per_batch / self.n_exp)
        capacity += capacity % 2 # make sure capacity is an even number
        capacity = max(capacity, self.min_capacity) # use min capacity
        assert capacity > 0
        return int(capacity)    
    """
    return capacity

In [17]:
math.floor(1*1.25*1024/9)

142

1